# Neurofeedback Model Demo

This notebook demonstrates the computational model of neurofeedback based on Davelaar (2018). We'll explore the main components of the model, run simulations, and visualize the results.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add the src directory to the path
src_dir = Path('..') / "src"
sys.path.append(str(src_dir))

# Import the neurofeedback model
from neurofeedback_model import NeurofeedbackModel
from brian2 import second, ms

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 1. Understanding the Model

The neurofeedback model consists of these key components:

1. **Spiking Neuron Network**: An Izhikevich-type neural network that generates EEG-like signals
2. **Striatal Learning Mechanism**: A reinforcement learning process that modifies the activation probabilities of striatal units
3. **Feedback System**: Real-time analysis of the EEG signal to provide feedback

According to Davelaar's theory, neurofeedback learning occurs in multiple stages, with the striatum playing a critical role in the initial exploration phase. The model implements this first stage of learning.

## 2. Setting Up the Model

Let's create an instance of the neurofeedback model with default parameters.

In [ ]:
# Create results directory
results_dir = Path("../results/notebook_demo")
results_dir.mkdir(parents=True, exist_ok=True)

# Set parameters
params = {
    'ne': 800,               # number of excitatory neurons
    'ni': 200,               # number of inhibitory neurons
    'n_striatum': 1000,      # number of striatal units
    'duration': 5*second,    # simulation duration (shorter for demo)
    'dt': 0.1*ms,            # simulation time step
    'feedback_interval': 100*ms,  # how often to give feedback
    'learning_rate': 0.01,   # learning rate for weights update
    'alpha_band': (8, 12),   # alpha frequency band (Hz)
    'threshold': 1.0,        # threshold for positive feedback
    'seed': 42,              # random seed for reproducibility
    'results_dir': str(results_dir)
}

# Create model
model = NeurofeedbackModel(**params)

print(f"Model created with {model.ne} excitatory neurons and {model.ni} inhibitory neurons")
print(f"Target MSN unit: {model.target}")

## 3. Running a Baseline Simulation

First, let's run a baseline simulation without neurofeedback learning to understand the natural distribution of alpha power.

In [ ]:
# Run baseline simulation
baseline_results = model.simulate(baseline_run=True)

# Extract results
alpha_power_history = baseline_results['results']['alpha_power_history']

# Calculate statistics
mean_alpha = np.mean(alpha_power_history)
std_alpha = np.std(alpha_power_history)
min_alpha = np.min(alpha_power_history)
max_alpha = np.max(alpha_power_history)

print(f"Baseline alpha power statistics:")
print(f"Mean: {mean_alpha:.4f}")
print(f"Std: {std_alpha:.4f}")
print(f"Min: {min_alpha:.4f}")
print(f"Max: {max_alpha:.4f}")

# Plot baseline alpha power
plt.figure(figsize=(10, 6))
plt.plot(alpha_power_history, label='Alpha Power')
plt.axhline(y=mean_alpha, color='r', linestyle='--', label=f'Mean ({mean_alpha:.4f})')
plt.xlabel('Feedback Index')
plt.ylabel('Alpha Power')
plt.title('Baseline Alpha Power')
plt.legend()
plt.grid(True)
plt.show()

# Plot histogram of baseline alpha power
plt.figure(figsize=(10, 6))
sns.histplot(alpha_power_history, kde=True)
plt.axvline(x=mean_alpha, color='r', linestyle='--', label=f'Mean ({mean_alpha:.4f})')
plt.xlabel('Alpha Power')
plt.ylabel('Frequency')
plt.title('Distribution of Baseline Alpha Power')
plt.legend()
plt.grid(True)
plt.show()

### Setting the threshold based on baseline

Let's set our threshold for positive feedback based on the mean of the baseline alpha power.

In [ ]:
# Set threshold to mean baseline alpha power
model.threshold = mean_alpha
print(f"Threshold set to {model.threshold:.4f}")

## 4. Running Neurofeedback Training

Now, let's run the actual neurofeedback training simulation and analyze the results.

In [ ]:
# Run neurofeedback training
results = model.simulate()

# Plot results
model.plot_results(results)

## 5. Analyzing the Learning Process

Let's analyze how the neurofeedback learning process unfolded.

In [ ]:
# Extract results data
weight_history = results['results']['weight_history']
alpha_power_history = results['results']['alpha_power_history']
feedback_history = results['results']['feedback_history']
target_activation_history = results['results']['target_activation_history']

# Calculate feedback window
feedback_window = 10
success_rate = np.convolve(feedback_history, np.ones(feedback_window)/feedback_window, mode='valid')

# Plot moving average of success rate
plt.figure(figsize=(10, 6))
plt.plot(success_rate, label=f'Success Rate (MA-{feedback_window})')
plt.xlabel('Feedback Index')
plt.ylabel('Success Rate')
plt.title('Moving Average of Positive Feedback Rate')
plt.legend()
plt.grid(True)
plt.show()

# Plot target activations vs feedback
plt.figure(figsize=(10, 6))
plt.plot(target_activation_history, 'b-', label='Target MSN Active', alpha=0.5)
plt.plot(feedback_history, 'g-', label='Positive Feedback', alpha=0.5)
plt.xlabel('Feedback Index')
plt.ylabel('Status (0=False, 1=True)')
plt.title('Target MSN Activation vs Positive Feedback')
plt.legend()
plt.grid(True)
plt.show()

# Calculate correlation between target activation and feedback
correlation = np.corrcoef(target_activation_history, feedback_history)[0, 1]
print(f"Correlation between target activation and positive feedback: {correlation:.4f}")

# Calculate the final success metrics
final_target_weight = weight_history[-1]
final_alpha_power = alpha_power_history[-1]
overall_success_rate = sum(feedback_history) / len(feedback_history)

print(f"Final target MSN weight: {final_target_weight:.4f}")
print(f"Final alpha power: {final_alpha_power:.4f}")
print(f"Overall success rate: {overall_success_rate:.2f}")

## 6. Comparing Alpha Power Distributions

Let's compare the distribution of alpha power before and after neurofeedback training to see how it changed.

In [ ]:
# Get baseline and training alpha power
baseline_alpha = baseline_results['results']['alpha_power_history']
training_alpha = results['results']['alpha_power_history']

# Plot distributions
plt.figure(figsize=(12, 6))
sns.kdeplot(baseline_alpha, label='Baseline', fill=True, alpha=0.5)
sns.kdeplot(training_alpha, label='After Training', fill=True, alpha=0.5)
plt.axvline(x=np.mean(baseline_alpha), color='blue', linestyle='--', 
            label=f'Baseline Mean ({np.mean(baseline_alpha):.4f})')
plt.axvline(x=np.mean(training_alpha), color='orange', linestyle='--', 
            label=f'Training Mean ({np.mean(training_alpha):.4f})')
plt.xlabel('Alpha Power')
plt.ylabel('Density')
plt.title('Alpha Power Distribution Before and After Training')
plt.legend()
plt.grid(True)
plt.show()

# Calculate percent change
percent_change = (np.mean(training_alpha) - np.mean(baseline_alpha)) / np.mean(baseline_alpha) * 100
print(f"Mean alpha power increased by {percent_change:.2f}%")

## 7. Parameter Exploration

Let's explore the effect of different learning rates on neurofeedback training.

In [ ]:
# Try different learning rates
learning_rates = [0.001, 0.005, 0.01, 0.02, 0.05]
final_weights = []
final_powers = []
success_rates = []

for lr in learning_rates:
    print(f"\nRunning simulation with learning_rate = {lr}")
    
    # Create a new model with this learning rate
    params['learning_rate'] = lr
    params['seed'] = 42  # Keep same seed for reproducibility
    test_model = NeurofeedbackModel(**params)
    
    # Run simulation
    test_results = test_model.simulate()
    
    # Extract metrics
    final_target_weight = test_results['results']['weight_history'][-1]
    final_alpha_power = test_results['results']['alpha_power_history'][-1]
    overall_success_rate = sum(test_results['results']['feedback_history']) / len(test_results['results']['feedback_history'])
    
    print(f"Final target weight: {final_target_weight:.4f}")
    print(f"Final alpha power: {final_alpha_power:.4f}")
    print(f"Success rate: {overall_success_rate:.2f}")
    
    final_weights.append(final_target_weight)
    final_powers.append(final_alpha_power)
    success_rates.append(overall_success_rate)

# Plot results
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

# Plot final weights
axs[0].plot(learning_rates, final_weights, 'o-')
axs[0].set_ylabel('Final Target Weight')
axs[0].set_title('Effect of Learning Rate on Final Target Weight')
axs[0].grid(True)

# Plot final alpha power
axs[1].plot(learning_rates, final_powers, 'o-')
axs[1].set_ylabel('Final Alpha Power')
axs[1].set_title('Effect of Learning Rate on Final Alpha Power')
axs[1].grid(True)

# Plot success rates
axs[2].plot(learning_rates, success_rates, 'o-')
axs[2].set_ylabel('Success Rate')
axs[2].set_xlabel('Learning Rate')
axs[2].set_title('Effect of Learning Rate on Success Rate')
axs[2].grid(True)

plt.tight_layout()
plt.show()

## 8. Optimal Threshold Setting

Let's investigate the effect of different threshold settings on learning.

In [ ]:
# Try different threshold multipliers relative to baseline mean
threshold_multipliers = [0.8, 0.9, 1.0, 1.1, 1.2]
thresholds = [mean_alpha * mult for mult in threshold_multipliers]
final_weights = []
final_powers = []
success_rates = []

for threshold in thresholds:
    print(f"\nRunning simulation with threshold = {threshold:.4f}")
    
    # Create a new model with this threshold
    params['threshold'] = threshold
    params['learning_rate'] = 0.01  # Reset to default
    params['seed'] = 42  # Keep same seed for reproducibility
    test_model = NeurofeedbackModel(**params)
    
    # Run simulation
    test_results = test_model.simulate()
    
    # Extract metrics
    final_target_weight = test_results['results']['weight_history'][-1]
    final_alpha_power = test_results['results']['alpha_power_history'][-1]
    overall_success_rate = sum(test_results['results']['feedback_history']) / len(test_results['results']['feedback_history'])
    
    print(f"Final target weight: {final_target_weight:.4f}")
    print(f"Final alpha power: {final_alpha_power:.4f}")
    print(f"Success rate: {overall_success_rate:.2f}")
    
    final_weights.append(final_target_weight)
    final_powers.append(final_alpha_power)
    success_rates.append(overall_success_rate)

# Plot results
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

# Plot final weights
axs[0].plot(threshold_multipliers, final_weights, 'o-')
axs[0].set_ylabel('Final Target Weight')
axs[0].set_title('Effect of Threshold Multiplier on Final Target Weight')
axs[0].grid(True)

# Plot final alpha power
axs[1].plot(threshold_multipliers, final_powers, 'o-')
axs[1].set_ylabel('Final Alpha Power')
axs[1].set_title('Effect of Threshold Multiplier on Final Alpha Power')
axs[1].grid(True)

# Plot success rates
axs[2].plot(threshold_multipliers, success_rates, 'o-')
axs[2].set_ylabel('Success Rate')
axs[2].set_xlabel('Threshold Multiplier (relative to baseline mean)')
axs[2].set_title('Effect of Threshold Multiplier on Success Rate')
axs[2].grid(True)

plt.tight_layout()
plt.show()

## 9. Conclusion

In this notebook, we've explored the neurofeedback computational model based on Davelaar's theory. We have:

1. Set up the model and run baseline and training simulations
2. Analyzed the learning process and how it affects alpha power
3. Compared alpha power distributions before and after training
4. Explored the effects of different learning rates and thresholds on training effectiveness

The model provides a computational framework for understanding the neural mechanisms of neurofeedback training, particularly the role of the striatum in the initial learning phase.

Key findings from our parameter exploration:
- Learning rate affects the speed of convergence, with an optimal value likely between 0.005 and 0.02
- Threshold setting is critical for successful learning, with the optimal threshold typically near the mean of the baseline alpha power

These insights can help guide the design of more effective neurofeedback protocols in clinical applications.